# <center>Лабораторная работа №2<center>

Выполнила студентка группы 8О-307Б \
Усачева Елизавета

## Задание

Необходимо реализовать алгоритмы машинного обучения:
* ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ
* KNN
* SVM
* ДЕРЕВО РЕШЕНИЙ
* RANDOM FOREST

Применить данные алгоритмы на наборы данных, подготовленных в первой лабораторной работе. \
Провести анализ полученных моделей, вычислить метрики классификатора. \
Произвести тюнинг параметров в случае необходимости. \
Сравнить полученные результаты с моделями, реализованными в scikit-learn. \
Аналогично построить метрики классификации. \
Показать, что полученные модели не переобучились. \
Также необходимо сделать выводы о применимости данных моделей к вашей задаче.

## Подготовка данных

Тестовая и тренировочная выборки были подготовлены в Лабораторной работе №1 (первый датасет).

In [63]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

Импортируем данные:

In [64]:
data_train = pd.read_csv("data/data_train1.csv")
data_test = pd.read_csv("data/data_test1.csv")

In [65]:
data_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
data_test.drop(['Unnamed: 0'], axis = 1, inplace = True)

Перед обучением выделим целевую переменную и нормализуем данные.

In [66]:
y_train = data_train['Purchased'].to_numpy()
y_test = data_test['Purchased'].to_numpy()

x_train = data_train.drop('Purchased', axis=1)
x_test = data_test.drop('Purchased', axis=1)

In [67]:
scale_features_std = MinMaxScaler()

x_train = scale_features_std.fit_transform(x_train.to_numpy()) 
x_test = scale_features_std.transform(x_test.to_numpy())

## Модели

### Логистическая регрессия

Логистическая регрессия или логит-модель (англ. logit model) — это статистическая модель, используемая для прогнозирования вероятности возникновения некоторого события путём его сравнения с логистической кривой. Эта регреcсия выдаёт ответ в виде вероятности бинарного события (1 или 0).

* Логистическая регрессия — одно из статистических методов классификации с использованием линейного дискриминанта Фишера.
* Значением функции является вероятность того, что данное исходное значение принадлежит к определенному классу.
* Механизм обучения логистической регрессии старается максимизировать среднее значение g(x).

#### Реализация

В качестве оптимизатора используется метод градиентного супска с постоянным шагом, в качестве метрики в этом и последующих методах выступает precision.

In [68]:
def logistic_func(beta, X):
    return 1.0/(1 + np.exp(-np.dot(X, beta.T))) 

def log_gradient(beta, X, y):
    first_calc = logistic_func(beta, X) - y.reshape(X.shape[0], -1) 
    final_calc = np.dot(first_calc.T, X) 
    return final_calc 

def cost_func(beta, X, y): 
    log_func_v = logistic_func(beta, X) 
    y = np.squeeze(y) 
    step1 = y * np.log(log_func_v) 
    step2 = (1 - y) * np.log(1 - log_func_v) 
    final = -step1 - step2 
    return np.mean(final) 

def grad_desc(X, y, beta, lr=.01, converge_change=.001): 
    cost = cost_func(beta, X, y) 
    change_cost = 1
    while(change_cost > converge_change): 
        old_cost = cost 
        beta = beta - (lr * log_gradient(beta, X, y)) 
        cost = cost_func(beta, X, y) 
        change_cost = old_cost - cost
    return beta

def pred_values(beta, X): 
    pred_prob = logistic_func(beta, X) 
    pred_value = np.where(pred_prob >= .5, 1, 0) 
    return np.squeeze(pred_value)

#### Реализация из scikit-learn

In [69]:
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### Результаты работы

Реализация:

In [70]:
beta = np.matrix(np.zeros(x_train.shape[1]))
beta = grad_desc(x_train, y_train, beta)

print("train precision: " + str(precision_score(y_train, pred_values(beta, x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, pred_values(beta, x_test), average='weighted')))

train precision: 0.5708525458168887
test precision: 0.5160881801125704


Реализация из scikit-learn:

In [71]:
print("train precision: " + str(precision_score(y_train, log_reg.predict(x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, log_reg.predict(x_test), average='weighted')))

train precision: 0.842041015625
test precision: 0.8358490566037735


Метрика моей реализации реализации оказалась очень плохой. К тому же, результат на тренировочных данных оказался лучше, чем на тестовых, что может указывать на переобучение.

На реализации из scikit-learn метрика показала вполне приемлимые результаты, но тут так же присутсвует подозрение на переобучение.


### KNN

Метод k-ближайших соседей используется для решения задачи классификации. Он относит объекты к классу, которому принадлежит большинство из k его ближайших соседей в многомерном пространстве признаков.

#### Реализация

In [72]:
def SquareEuclidDistance(a,b):
    d = 0
    for i in range(a.shape[0]):
        d += (a[i] - b[i]) * (a[i] - b[i])
    return d

def KNN(X_train, Y_train, X_test):
    Y_test = np.ones(X_test.shape[0])
    for j in range(X_test.shape[0]):
        Q = np.zeros(Y_train.max() + 1)
        for i in range(X_train.shape[0]):
            Q[Y_train[i]] += 1/SquareEuclidDistance(X_test[j,:], X_train[i,:])
        Y_test[j] = np.argmax(Q)
    return Y_test

#### Реализация из scikit-learn

In [73]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

#### Результаты работы

Реализация:

In [74]:
print("train precision: " + str(precision_score(y_test, KNN(x_test, y_test, x_test), average='weighted')))
print("test precision: " + str(precision_score(y_test, KNN(x_train, y_train, x_test), average='weighted')))

train precision: 1.0
test precision: 0.8687148217636022


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':


Реализация из scikit-learn:

In [75]:
print("train precision: " + str(precision_score(y_train, knn.predict(x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, knn.predict(x_test), average='weighted')))

train precision: 0.9112725948869678
test precision: 0.929513888888889


Точно можно сказать, что данный метод сработал лучше чем предыдущий. Однако, здесь так же видно, что в обоих реализациях модель переобучилась. На тестовой выборке реализация из scikit-learn показала более высокую точность.

### Дерево решений

Дерево решений — алгоритм классификации, задающийся деревом (связным ациклическим графом).

#### Реализация

In [76]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.feature_prun = 0.1

    def fit(self, X, y, random_feature = False):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y, random_feature)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    
    def _best_split(self, X, y,random_feature):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            if(np.random.randint(0, 11) <= self.feature_prun*10):
                continue
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y,random_feature, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y,random_feature)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left,random_feature, depth + 1)
                node.right = self._grow_tree(X_right, y_right,random_feature, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

#### Реализация из scikit-learn

In [77]:
dec_tree = tree.DecisionTreeClassifier()
dec_tree = dec_tree.fit(x_train, y_train)

#### Результаты работы

Реализация:

In [78]:
dec_tree2 = DecisionTreeClassifier(max_depth=10)
dec_tree2.fit(x_train, y_train)
print("train precision: " + str(precision_score(y_train, dec_tree2.predict(x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, dec_tree2.predict(x_test), average='weighted')))

train precision: 0.9907467532467532
test precision: 0.8152725563909776


Реализация из scikit-learn:

In [79]:
print("train precision: " + str(precision_score(y_train, dec_tree.predict(x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, dec_tree.predict(x_test), average='weighted')))

train precision: 0.9968886462882096
test precision: 0.8152725563909776


Обе реализации отработали хорошо.

### Random forest

RF (random forest) — это множество решающих деревьев. В задаче регрессии их ответы усредняются, в задаче классификации принимается решение голосованием по большинству. Все деревья строятся независимо по следующей схеме:

* Выбирается подвыборка обучающей выборки размера samplesize (м.б. с возвращением) – по ней строится дерево (для каждого дерева — своя подвыборка).

* Для построения каждого расщепления в дереве просматриваем max_features случайных признаков (для каждого нового расщепления — свои случайные признаки).

* Выбираем наилучшие признак и расщепление по нему (по заранее заданному критерию). Дерево строится, как правило, до исчерпания выборки (пока в листьях не останутся представители только одного класса), но в современных реализациях есть параметры, которые ограничивают высоту дерева, число объектов в листьях и число объектов в подвыборке, при котором проводится расщепление.

#### Реализация

In [80]:
def RandomForest(size, Max_depth):
    head = [None] * size
    for i in range(size):
        head[i] = DecisionTreeClassifier(max_depth=Max_depth)
    return head
def fit(forest, X_train, y_train):
    for i in range(len(forest)):
        subset = np.zeros(X_train.shape)
        labels = np.zeros(y_train.shape).astype(int)
        for j in range(X_train.shape[0]):
            index = np.random.randint(0, X_train.shape[0])
            subset[j] = X_train[index]
            labels[j] = y_train[index]
        forest[i].fit(X_train, y_train,random_feature = True)
def predict(forest, X):
    Q = np.zeros([X.shape[0], 2])
    for i in range(len(forest)):
        pred = forest[i].predict(X)
        for j in range(len(pred)):
            Q[j, pred[j]] += 1
    pred = np.zeros([X.shape[0]])
    for i in range(X.shape[0]):
        pred[i] = np.argmax(Q[i,:])
    return pred

#### Реализация из scikit-learn:

In [81]:
rf = RandomForestClassifier(max_depth=4)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Результаты работы

Реализация:

In [82]:
rf2 = RandomForest(10, 4)
fit(rf2, x_train, y_train)
print("train precision: " + str(precision_score(y_train, predict(rf2,x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, predict(rf2,x_test), average='weighted')))

train precision: 0.9274624530663329
test precision: 0.8764204545454545


Реализация из scikit-learn:

In [83]:
print("train precision: " + str(precision_score(y_train, rf.predict(x_train), average='weighted')))
print("test precision: " + str(precision_score(y_test, rf.predict(x_test), average='weighted')))

train precision: 0.9328480828598273
test precision: 0.9466911764705882


Модель из scikit-learn показала лучший результат, чем моя реализация. Также, на тестовой выборке, метод RF отработал лучше, чем дерево решений.

### Вывод

В данной лабораторной работе мы реализовали несколько базовых алгоритмов машинного обучения и сравнили их точность с алгоритмами, реализованными в библиотеке scikit-learn. Однако, во всех случаях модель переобучилась. Это может быть связано с маленьким коливеством признаков или образцов из датасета.